In [ ]:
import pandas as pd
from collections import Counter
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
import numpy as np

# RとPythonのデータフレームの相互変換を有効化
pandas2ri.activate()
rpy2.robjects.numpy2ri.activate()

# サンプルデータの作成
data = {
    'tokenized_body': [
        ['this', 'is', 'a', 'sample', 'document'],
        ['another', 'sample', 'document', 'with', 'different', 'words'],
        ['text', 'data', 'can', 'be', 'very', 'informative'],
        ['this', 'is', 'yet', 'another', 'example', 'of', 'a', 'document'],
        ['machine', 'learning', 'models', 'can', 'be', 'complex']
    ],
    'metadata': [
        'Category1',
        'Category2',
        'Category1',
        'Category2',
        'Category1'
    ]
}
df = pd.DataFrame(data)

# 語彙リストを作成し、インデックスが1から始まるようにする
vocab = sorted(list(set([word for doc in df['tokenized_body'] for word in doc])))  # ソートしてインデックスが連続するようにする
vocab_dict = {word: idx + 1 for idx, word in enumerate(vocab)}  # インデックスが1から始まるようにする

# 各文書をインデックスと頻度の行列に変換
documents = []
for doc in df['tokenized_body']:
    word_counts = Counter(doc)
    indices = [vocab_dict[word] for word in word_counts.keys()]
    counts = list(word_counts.values())
    documents.append(np.column_stack((indices, counts)))

# Rのリスト形式に変換
documents_r = ro.ListVector({'doc' + str(i+1): ro.r.matrix(np.array(doc), ncol=2) for i, doc in enumerate(documents)})

# RのSTMパッケージをインポート
stm = importr('stm')

# メタデータの変換（ここでは文字列として扱う）
metadata = pandas2ri.py2rpy(df[['metadata']].astype(str))

# トピックモデルの作成
model = stm.stm(documents=documents_r, vocab=vocab, K=3, prevalence=metadata, data=metadata)
